In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import Adam, SGD
import torchvision
import torchvision.transforms as transforms

import sys, os, math
import argparse

In [2]:
lr=0.01
data='cifar10'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar10_vgg_ProActiv.pth'
resume = False

In [3]:
if data == 'cifar10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'cifar100':
    nclass = 100
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR100(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR100(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'stl10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.STL10(root=root, split='train', transform=transform_train, target_transform=None, download=True)
    testset = torchvision.datasets.STL10(root=root, split='test', transform=transform_test, target_transform=None, download=True)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
if model == 'vgg':
    from models.vgg_dist import VGG_Dist
    net = nn.DataParallel(VGG_Dist(nclass, img_width=img_width).cuda())
    
net

DataParallel(
  (module): VGG_Dist(
    (conv1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv4): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    

In [5]:
if resume:
    print(f'==> Resuming from {model_out}')
    net.load_state_dict(torch.load(model_out))

In [6]:
cudnn.benchmark = True

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
def train(epoch):
    print('Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        pred = torch.max(outputs, dim=1)[1]
        correct += torch.sum(pred.eq(targets)).item()
        total += targets.numel()
    print(f'[TRAIN] Acc: {100.*correct/total:.3f}')

In [9]:
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'[TEST] Acc: {100.*correct/total:.3f}')

    # Save checkpoint after each epoch
    torch.save(net.state_dict(), model_out)

In [10]:
if data == 'cifar10':
    epochs = [50, 50, 50, 50]
elif data == 'cifar100':
    epochs = [50, 50, 50, 50]
elif data == 'stl10':
    epochs = [50, 50, 50, 50]

In [11]:
count = 0

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 14.850
[TEST] Acc: 16.190
Epoch: 1
[TRAIN] Acc: 17.494
[TEST] Acc: 16.750
Epoch: 2
[TRAIN] Acc: 18.588
[TEST] Acc: 22.250
Epoch: 3
[TRAIN] Acc: 21.028
[TEST] Acc: 22.350
Epoch: 4
[TRAIN] Acc: 24.364
[TEST] Acc: 27.290
Epoch: 5
[TRAIN] Acc: 26.538
[TEST] Acc: 29.720
Epoch: 6
[TRAIN] Acc: 30.364
[TEST] Acc: 31.840
Epoch: 7
[TRAIN] Acc: 33.790
[TEST] Acc: 45.220
Epoch: 8
[TRAIN] Acc: 42.310
[TEST] Acc: 44.040
Epoch: 9
[TRAIN] Acc: 49.844
[TEST] Acc: 51.660
Epoch: 10
[TRAIN] Acc: 54.200
[TEST] Acc: 51.750
Epoch: 11
[TRAIN] Acc: 57.168
[TEST] Acc: 57.040
Epoch: 12
[TRAIN] Acc: 59.622
[TEST] Acc: 68.020
Epoch: 13
[TRAIN] Acc: 62.484
[TEST] Acc: 68.400
Epoch: 14
[TRAIN] Acc: 65.564
[TEST] Acc: 67.030
Epoch: 15
[TRAIN] Acc: 68.110
[TEST] Acc: 68.210
Epoch: 16
[TRAIN] Acc: 70.466
[TEST] Acc: 72.060
Epoch: 17
[TRAIN] Acc: 72.238
[TEST] Acc: 72.670
Epoch: 18
[TRAIN] Acc: 74.224
[TEST] Acc: 76.260
Epoch: 19
[TRAIN] Acc: 75.978
[TEST] Acc: 77.330
Epoch: 20
[TRAIN] Acc: 77.494
[TEST] Ac

[TEST] Acc: 87.880
Epoch: 167
[TRAIN] Acc: 99.500
[TEST] Acc: 87.870
Epoch: 168
[TRAIN] Acc: 99.544
[TEST] Acc: 87.480
Epoch: 169
[TRAIN] Acc: 99.502
[TEST] Acc: 87.660
Epoch: 170
[TRAIN] Acc: 99.534
[TEST] Acc: 87.670
Epoch: 171
[TRAIN] Acc: 99.518
[TEST] Acc: 87.540
Epoch: 172
[TRAIN] Acc: 99.504
[TEST] Acc: 87.750
Epoch: 173
[TRAIN] Acc: 99.582
[TEST] Acc: 87.690
Epoch: 174
[TRAIN] Acc: 99.500
[TEST] Acc: 87.690
Epoch: 175
[TRAIN] Acc: 99.488
[TEST] Acc: 87.650
Epoch: 176
[TRAIN] Acc: 99.472
[TEST] Acc: 87.670
Epoch: 177
[TRAIN] Acc: 99.506
[TEST] Acc: 87.710
Epoch: 178
[TRAIN] Acc: 99.526
[TEST] Acc: 87.410
Epoch: 179
[TRAIN] Acc: 99.558
[TEST] Acc: 87.540
Epoch: 180
[TRAIN] Acc: 99.524
[TEST] Acc: 87.420
Epoch: 181
[TRAIN] Acc: 99.540
[TEST] Acc: 87.430
Epoch: 182
[TRAIN] Acc: 99.496
[TEST] Acc: 87.950
Epoch: 183
[TRAIN] Acc: 99.514
[TEST] Acc: 87.750
Epoch: 184
[TRAIN] Acc: 99.524
[TEST] Acc: 87.610
Epoch: 185
[TRAIN] Acc: 99.554
[TEST] Acc: 87.790
Epoch: 186
[TRAIN] Acc: 99.556


In [2]:
lr=0.01
data='stl10'
root='./data/'
model='vgg'
model_out='./checkpoint/stl10_vgg_ProActiv.pth'
resume = False

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 10.060
[TEST] Acc: 10.100
Epoch: 1
[TRAIN] Acc: 9.660
[TEST] Acc: 9.912
Epoch: 2
[TRAIN] Acc: 9.960
[TEST] Acc: 10.675
Epoch: 3
[TRAIN] Acc: 10.320
[TEST] Acc: 9.688
Epoch: 4
[TRAIN] Acc: 9.920
[TEST] Acc: 9.537
Epoch: 5
[TRAIN] Acc: 10.580
[TEST] Acc: 15.825
Epoch: 6
[TRAIN] Acc: 12.640
[TEST] Acc: 14.312
Epoch: 7
[TRAIN] Acc: 14.900
[TEST] Acc: 17.238
Epoch: 8
[TRAIN] Acc: 15.200
[TEST] Acc: 16.125
Epoch: 9
[TRAIN] Acc: 14.120
[TEST] Acc: 19.600
Epoch: 10
[TRAIN] Acc: 16.160
[TEST] Acc: 19.337
Epoch: 11
[TRAIN] Acc: 16.960
[TEST] Acc: 18.488
Epoch: 12
[TRAIN] Acc: 17.540
[TEST] Acc: 14.750
Epoch: 13
[TRAIN] Acc: 18.120
[TEST] Acc: 18.375
Epoch: 14
[TRAIN] Acc: 18.360
[TEST] Acc: 17.700
Epoch: 15
[TRAIN] Acc: 17.420
[TEST] Acc: 15.988
Epoch: 16
[TRAIN] Acc: 18.280
[TEST] Acc: 19.975
Epoch: 17
[TRAIN] Acc: 18.460
[TEST] Acc: 19.637
Epoch: 18
[TRAIN] Acc: 17.100
[TEST] Acc: 21.325
Epoch: 19
[TRAIN] Acc: 18.460
[TEST] Acc: 22.163
Epoch: 20
[TRAIN] Acc: 17.820
[TEST] Acc: 23.

[TEST] Acc: 35.575
Epoch: 167
[TRAIN] Acc: 37.460
[TEST] Acc: 35.175
Epoch: 168
[TRAIN] Acc: 37.920
[TEST] Acc: 35.400
Epoch: 169
[TRAIN] Acc: 36.500
[TEST] Acc: 35.188
Epoch: 170
[TRAIN] Acc: 36.640
[TEST] Acc: 35.525
Epoch: 171
[TRAIN] Acc: 37.420
[TEST] Acc: 34.725
Epoch: 172
[TRAIN] Acc: 37.780
[TEST] Acc: 34.525
Epoch: 173
[TRAIN] Acc: 36.640
[TEST] Acc: 34.725
Epoch: 174
[TRAIN] Acc: 37.480
[TEST] Acc: 35.312
Epoch: 175
[TRAIN] Acc: 37.520
[TEST] Acc: 34.987
Epoch: 176
[TRAIN] Acc: 37.580
[TEST] Acc: 35.587
Epoch: 177
[TRAIN] Acc: 37.780
[TEST] Acc: 35.175
Epoch: 178
[TRAIN] Acc: 38.060
[TEST] Acc: 34.725
Epoch: 179
[TRAIN] Acc: 38.480
[TEST] Acc: 35.087
Epoch: 180
[TRAIN] Acc: 37.160
[TEST] Acc: 35.150
Epoch: 181
[TRAIN] Acc: 37.680
[TEST] Acc: 34.562
Epoch: 182
[TRAIN] Acc: 38.140
[TEST] Acc: 34.625
Epoch: 183
[TRAIN] Acc: 37.740
[TEST] Acc: 34.763
Epoch: 184
[TRAIN] Acc: 36.260
[TEST] Acc: 34.600
Epoch: 185
[TRAIN] Acc: 38.620
[TEST] Acc: 35.025
Epoch: 186
[TRAIN] Acc: 36.180


In [2]:
lr=0.01
data='cifar100'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar100_vgg_ProActiv.pth'
resume = False

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 0.990
[TEST] Acc: 1.120
Epoch: 1
[TRAIN] Acc: 0.926
[TEST] Acc: 1.070
Epoch: 2
[TRAIN] Acc: 1.008
[TEST] Acc: 1.130
Epoch: 3
[TRAIN] Acc: 0.932
[TEST] Acc: 1.090
Epoch: 4
[TRAIN] Acc: 1.016
[TEST] Acc: 0.990
Epoch: 5
[TRAIN] Acc: 0.976
[TEST] Acc: 1.060
Epoch: 6
[TRAIN] Acc: 1.040
[TEST] Acc: 1.030
Epoch: 7
[TRAIN] Acc: 1.078
[TEST] Acc: 0.950
Epoch: 8
[TRAIN] Acc: 0.984
[TEST] Acc: 0.960
Epoch: 9
[TRAIN] Acc: 1.032
[TEST] Acc: 0.890
Epoch: 10
[TRAIN] Acc: 0.966
[TEST] Acc: 0.880
Epoch: 11
[TRAIN] Acc: 1.006
[TEST] Acc: 1.000
Epoch: 12
[TRAIN] Acc: 1.060
[TEST] Acc: 0.980
Epoch: 13
[TRAIN] Acc: 0.954
[TEST] Acc: 0.970
Epoch: 14
[TRAIN] Acc: 1.016
[TEST] Acc: 1.030
Epoch: 15
[TRAIN] Acc: 1.034
[TEST] Acc: 0.970
Epoch: 16
[TRAIN] Acc: 0.974
[TEST] Acc: 1.060
Epoch: 17
[TRAIN] Acc: 1.028
[TEST] Acc: 0.950
Epoch: 18
[TRAIN] Acc: 0.958
[TEST] Acc: 1.040
Epoch: 19
[TRAIN] Acc: 0.964
[TEST] Acc: 1.120
Epoch: 20
[TRAIN] Acc: 0.978
[TEST] Acc: 1.030
Epoch: 21
[TRAIN] Acc: 0.994
[TE

[TRAIN] Acc: 0.998
[TEST] Acc: 1.110
Epoch: 174
[TRAIN] Acc: 0.970
[TEST] Acc: 1.170
Epoch: 175
[TRAIN] Acc: 0.942
[TEST] Acc: 0.980
Epoch: 176
[TRAIN] Acc: 0.902
[TEST] Acc: 0.930
Epoch: 177
[TRAIN] Acc: 0.982
[TEST] Acc: 0.990
Epoch: 178
[TRAIN] Acc: 1.062
[TEST] Acc: 0.960
Epoch: 179
[TRAIN] Acc: 1.002
[TEST] Acc: 1.040
Epoch: 180
[TRAIN] Acc: 0.994
[TEST] Acc: 1.010
Epoch: 181
[TRAIN] Acc: 0.988
[TEST] Acc: 0.830
Epoch: 182
[TRAIN] Acc: 1.060
[TEST] Acc: 1.130
Epoch: 183
[TRAIN] Acc: 0.982
[TEST] Acc: 1.000
Epoch: 184
[TRAIN] Acc: 0.944
[TEST] Acc: 1.130
Epoch: 185
[TRAIN] Acc: 0.948
[TEST] Acc: 0.900
Epoch: 186
[TRAIN] Acc: 0.930
[TEST] Acc: 1.110
Epoch: 187
[TRAIN] Acc: 1.004
[TEST] Acc: 0.910
Epoch: 188
[TRAIN] Acc: 1.012
[TEST] Acc: 1.090
Epoch: 189
[TRAIN] Acc: 0.980
[TEST] Acc: 0.900
Epoch: 190
[TRAIN] Acc: 0.966
[TEST] Acc: 0.990
Epoch: 191
[TRAIN] Acc: 1.028
[TEST] Acc: 1.020
Epoch: 192
[TRAIN] Acc: 0.944
[TEST] Acc: 0.910
Epoch: 193
[TRAIN] Acc: 0.988
[TEST] Acc: 0.960
Epo

### f(x) = mu + w * eps

In [2]:
lr=0.01
data='cifar10'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar10_no_sigW_vgg_ProActiv.pth'
resume = False

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 10.682
[TEST] Acc: 11.900
Epoch: 1
[TRAIN] Acc: 19.810
[TEST] Acc: 26.790
Epoch: 2
[TRAIN] Acc: 37.716
[TEST] Acc: 44.570
Epoch: 3
[TRAIN] Acc: 55.194
[TEST] Acc: 46.300
Epoch: 4
[TRAIN] Acc: 66.022
[TEST] Acc: 66.180
Epoch: 5
[TRAIN] Acc: 72.302
[TEST] Acc: 68.440
Epoch: 6
[TRAIN] Acc: 76.516
[TEST] Acc: 73.100
Epoch: 7
[TRAIN] Acc: 79.802
[TEST] Acc: 74.860
Epoch: 8
[TRAIN] Acc: 82.470
[TEST] Acc: 75.280
Epoch: 9
[TRAIN] Acc: 84.760
[TEST] Acc: 77.770
Epoch: 10
[TRAIN] Acc: 86.724
[TEST] Acc: 80.740
Epoch: 11
[TRAIN] Acc: 88.312
[TEST] Acc: 79.010
Epoch: 12
[TRAIN] Acc: 89.494
[TEST] Acc: 80.060
Epoch: 13
[TRAIN] Acc: 90.772
[TEST] Acc: 79.650
Epoch: 14
[TRAIN] Acc: 92.080
[TEST] Acc: 82.030
Epoch: 15
[TRAIN] Acc: 92.856
[TEST] Acc: 80.980
Epoch: 16
[TRAIN] Acc: 93.644
[TEST] Acc: 76.700
Epoch: 17
[TRAIN] Acc: 94.366
[TEST] Acc: 81.730
Epoch: 18
[TRAIN] Acc: 94.964
[TEST] Acc: 80.490
Epoch: 19
[TRAIN] Acc: 95.502
[TEST] Acc: 81.270
Epoch: 20
[TRAIN] Acc: 96.002


[TEST] Acc: 86.980
Epoch: 166
[TRAIN] Acc: 100.000
[TEST] Acc: 86.970
Epoch: 167
[TRAIN] Acc: 100.000
[TEST] Acc: 86.960
Epoch: 168
[TRAIN] Acc: 100.000
[TEST] Acc: 86.870
Epoch: 169
[TRAIN] Acc: 100.000
[TEST] Acc: 87.030
Epoch: 170
[TRAIN] Acc: 99.998
[TEST] Acc: 86.970
Epoch: 171
[TRAIN] Acc: 100.000
[TEST] Acc: 87.010
Epoch: 172
[TRAIN] Acc: 100.000
[TEST] Acc: 86.960
Epoch: 173
[TRAIN] Acc: 99.998
[TEST] Acc: 87.000
Epoch: 174
[TRAIN] Acc: 100.000
[TEST] Acc: 86.990
Epoch: 175
[TRAIN] Acc: 99.998
[TEST] Acc: 87.010
Epoch: 176
[TRAIN] Acc: 100.000
[TEST] Acc: 86.960
Epoch: 177
[TRAIN] Acc: 100.000
[TEST] Acc: 86.940
Epoch: 178
[TRAIN] Acc: 100.000
[TEST] Acc: 87.010
Epoch: 179
[TRAIN] Acc: 100.000
[TEST] Acc: 87.060
Epoch: 180
[TRAIN] Acc: 100.000
[TEST] Acc: 87.090
Epoch: 181
[TRAIN] Acc: 100.000
[TEST] Acc: 86.950
Epoch: 182
[TRAIN] Acc: 100.000
[TEST] Acc: 86.940
Epoch: 183
[TRAIN] Acc: 100.000
[TEST] Acc: 87.040
Epoch: 184
[TRAIN] Acc: 100.000
[TEST] Acc: 87.130
Epoch: 185
[TRA

In [2]:
lr=0.01
data='cifar100'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar100_no_sigW_vgg_ProActiv.pth'
resume = False

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 1.558
[TEST] Acc: 2.280
Epoch: 1
[TRAIN] Acc: 2.722
[TEST] Acc: 3.260
Epoch: 2
[TRAIN] Acc: 3.684
[TEST] Acc: 4.550
Epoch: 3
[TRAIN] Acc: 5.172
[TEST] Acc: 5.520
Epoch: 4
[TRAIN] Acc: 6.050
[TEST] Acc: 7.060
Epoch: 5
[TRAIN] Acc: 7.580
[TEST] Acc: 11.340
Epoch: 6
[TRAIN] Acc: 9.294
[TEST] Acc: 13.300
Epoch: 7
[TRAIN] Acc: 11.252
[TEST] Acc: 13.630
Epoch: 8
[TRAIN] Acc: 13.720
[TEST] Acc: 12.990
Epoch: 9
[TRAIN] Acc: 16.918
[TEST] Acc: 18.170
Epoch: 10
[TRAIN] Acc: 19.080
[TEST] Acc: 18.580
Epoch: 11
[TRAIN] Acc: 21.896
[TEST] Acc: 21.250
Epoch: 12
[TRAIN] Acc: 24.292
[TEST] Acc: 21.410
Epoch: 13
[TRAIN] Acc: 27.640
[TEST] Acc: 26.930
Epoch: 14
[TRAIN] Acc: 29.918
[TEST] Acc: 27.450
Epoch: 15
[TRAIN] Acc: 32.416
[TEST] Acc: 30.970
Epoch: 16
[TRAIN] Acc: 34.816
[TEST] Acc: 34.700
Epoch: 17
[TRAIN] Acc: 37.522
[TEST] Acc: 36.040
Epoch: 18
[TRAIN] Acc: 39.274
[TEST] Acc: 32.600
Epoch: 19
[TRAIN] Acc: 41.944
[TEST] Acc: 37.290
Epoch: 20
[TRAIN] Acc: 45.060
[TEST] Acc: 

[TEST] Acc: 53.140
Epoch: 167
[TRAIN] Acc: 99.520
[TEST] Acc: 53.070
Epoch: 168
[TRAIN] Acc: 99.534
[TEST] Acc: 53.150
Epoch: 169
[TRAIN] Acc: 99.548
[TEST] Acc: 53.200
Epoch: 170
[TRAIN] Acc: 99.550
[TEST] Acc: 53.130
Epoch: 171
[TRAIN] Acc: 99.548
[TEST] Acc: 53.250
Epoch: 172
[TRAIN] Acc: 99.538
[TEST] Acc: 53.220
Epoch: 173
[TRAIN] Acc: 99.568
[TEST] Acc: 53.050
Epoch: 174
[TRAIN] Acc: 99.554
[TEST] Acc: 53.140
Epoch: 175
[TRAIN] Acc: 99.580
[TEST] Acc: 52.920
Epoch: 176
[TRAIN] Acc: 99.570
[TEST] Acc: 52.960
Epoch: 177
[TRAIN] Acc: 99.580
[TEST] Acc: 52.910
Epoch: 178
[TRAIN] Acc: 99.520
[TEST] Acc: 53.150
Epoch: 179
[TRAIN] Acc: 99.542
[TEST] Acc: 53.170
Epoch: 180
[TRAIN] Acc: 99.544
[TEST] Acc: 53.160
Epoch: 181
[TRAIN] Acc: 99.534
[TEST] Acc: 53.130
Epoch: 182
[TRAIN] Acc: 99.612
[TEST] Acc: 52.950
Epoch: 183
[TRAIN] Acc: 99.578
[TEST] Acc: 53.150
Epoch: 184
[TRAIN] Acc: 99.586
[TEST] Acc: 52.860
Epoch: 185
[TRAIN] Acc: 99.568
[TEST] Acc: 53.040
Epoch: 186
[TRAIN] Acc: 99.494


In [2]:
lr=0.01
data='stl10'
root='./data/'
model='vgg'
model_out='./checkpoint/stl10_no_SigW_vgg_ProActiv.pth'
resume = False

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 11.320
[TEST] Acc: 10.863
Epoch: 1
[TRAIN] Acc: 11.180
[TEST] Acc: 10.050
Epoch: 2
[TRAIN] Acc: 11.620
[TEST] Acc: 14.525
Epoch: 3
[TRAIN] Acc: 13.000
[TEST] Acc: 10.000
Epoch: 4
[TRAIN] Acc: 15.800
[TEST] Acc: 16.275
Epoch: 5
[TRAIN] Acc: 17.920
[TEST] Acc: 10.088
Epoch: 6
[TRAIN] Acc: 18.220
[TEST] Acc: 19.300
Epoch: 7
[TRAIN] Acc: 19.240
[TEST] Acc: 15.387
Epoch: 8
[TRAIN] Acc: 20.840
[TEST] Acc: 12.350
Epoch: 9
[TRAIN] Acc: 23.080
[TEST] Acc: 19.750
Epoch: 10
[TRAIN] Acc: 23.640
[TEST] Acc: 24.288
Epoch: 11
[TRAIN] Acc: 24.880
[TEST] Acc: 21.000
Epoch: 12
[TRAIN] Acc: 27.060
[TEST] Acc: 23.950
Epoch: 13
[TRAIN] Acc: 28.860
[TEST] Acc: 28.150
Epoch: 14
[TRAIN] Acc: 30.360
[TEST] Acc: 30.800
Epoch: 15
[TRAIN] Acc: 31.220
[TEST] Acc: 24.350
Epoch: 16
[TRAIN] Acc: 32.720
[TEST] Acc: 34.062
Epoch: 17
[TRAIN] Acc: 34.840
[TEST] Acc: 36.438
Epoch: 18
[TRAIN] Acc: 35.200
[TEST] Acc: 32.888
Epoch: 19
[TRAIN] Acc: 37.360
[TEST] Acc: 38.450
Epoch: 20
[TRAIN] Acc: 40.060


[TRAIN] Acc: 100.000
[TEST] Acc: 61.288
Epoch: 166
[TRAIN] Acc: 100.000
[TEST] Acc: 61.175
Epoch: 167
[TRAIN] Acc: 100.000
[TEST] Acc: 61.175
Epoch: 168
[TRAIN] Acc: 100.000
[TEST] Acc: 61.400
Epoch: 169
[TRAIN] Acc: 100.000
[TEST] Acc: 61.288
Epoch: 170
[TRAIN] Acc: 100.000
[TEST] Acc: 61.237
Epoch: 171
[TRAIN] Acc: 100.000
[TEST] Acc: 61.288
Epoch: 172
[TRAIN] Acc: 99.980
[TEST] Acc: 61.388
Epoch: 173
[TRAIN] Acc: 100.000
[TEST] Acc: 61.375
Epoch: 174
[TRAIN] Acc: 99.980
[TEST] Acc: 61.225
Epoch: 175
[TRAIN] Acc: 100.000
[TEST] Acc: 61.250
Epoch: 176
[TRAIN] Acc: 100.000
[TEST] Acc: 61.525
Epoch: 177
[TRAIN] Acc: 100.000
[TEST] Acc: 61.288
Epoch: 178
[TRAIN] Acc: 100.000
[TEST] Acc: 61.337
Epoch: 179
[TRAIN] Acc: 100.000
[TEST] Acc: 61.337
Epoch: 180
[TRAIN] Acc: 100.000
[TEST] Acc: 61.138
Epoch: 181
[TRAIN] Acc: 100.000
[TEST] Acc: 61.275
Epoch: 182
[TRAIN] Acc: 100.000
[TEST] Acc: 61.200
Epoch: 183
[TRAIN] Acc: 100.000
[TEST] Acc: 61.362
Epoch: 184
[TRAIN] Acc: 100.000
[TEST] Acc: 

### f(x) = mu + (alpha * sigmoid(mu) + beta) * eps

In [2]:
lr=0.01
data='cifar10'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar10_sigmoidX_vgg_ProActiv.pth'
resume = False

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 13.226
[TEST] Acc: 14.470
Epoch: 1
[TRAIN] Acc: 17.166
[TEST] Acc: 18.680
Epoch: 2
[TRAIN] Acc: 17.870
[TEST] Acc: 21.770
Epoch: 3
[TRAIN] Acc: 20.612
[TEST] Acc: 26.410
Epoch: 4
[TRAIN] Acc: 23.478
[TEST] Acc: 30.500
Epoch: 5
[TRAIN] Acc: 26.742
[TEST] Acc: 26.400
Epoch: 6
[TRAIN] Acc: 29.172
[TEST] Acc: 34.350
Epoch: 7
[TRAIN] Acc: 32.380
[TEST] Acc: 37.760
Epoch: 8
[TRAIN] Acc: 36.668
[TEST] Acc: 38.540
Epoch: 9
[TRAIN] Acc: 44.040
[TEST] Acc: 56.110
Epoch: 10
[TRAIN] Acc: 49.524
[TEST] Acc: 53.170
Epoch: 11
[TRAIN] Acc: 52.490
[TEST] Acc: 59.210
Epoch: 12
[TRAIN] Acc: 55.706
[TEST] Acc: 57.820
Epoch: 13
[TRAIN] Acc: 57.680
[TEST] Acc: 59.980
Epoch: 14
[TRAIN] Acc: 59.760
[TEST] Acc: 60.660
Epoch: 15
[TRAIN] Acc: 62.306
[TEST] Acc: 66.130
Epoch: 16
[TRAIN] Acc: 64.770
[TEST] Acc: 69.390
Epoch: 17
[TRAIN] Acc: 66.972
[TEST] Acc: 67.870
Epoch: 18
[TRAIN] Acc: 69.258
[TEST] Acc: 74.290
Epoch: 19
[TRAIN] Acc: 70.758
[TEST] Acc: 72.440
Epoch: 20
[TRAIN] Acc: 72.470
[TEST] Ac

[TEST] Acc: 87.420
Epoch: 167
[TRAIN] Acc: 98.880
[TEST] Acc: 86.900
Epoch: 168
[TRAIN] Acc: 98.826
[TEST] Acc: 87.190
Epoch: 169
[TRAIN] Acc: 98.878
[TEST] Acc: 87.230
Epoch: 170
[TRAIN] Acc: 98.798
[TEST] Acc: 86.990
Epoch: 171
[TRAIN] Acc: 98.862
[TEST] Acc: 87.410
Epoch: 172
[TRAIN] Acc: 98.780
[TEST] Acc: 87.280
Epoch: 173
[TRAIN] Acc: 98.788
[TEST] Acc: 87.050
Epoch: 174
[TRAIN] Acc: 98.862
[TEST] Acc: 87.380
Epoch: 175
[TRAIN] Acc: 98.818
[TEST] Acc: 87.190
Epoch: 176
[TRAIN] Acc: 98.868
[TEST] Acc: 87.210
Epoch: 177
[TRAIN] Acc: 98.760
[TEST] Acc: 87.630
Epoch: 178
[TRAIN] Acc: 98.922
[TEST] Acc: 87.130
Epoch: 179
[TRAIN] Acc: 98.866
[TEST] Acc: 86.990
Epoch: 180
[TRAIN] Acc: 98.808
[TEST] Acc: 87.250
Epoch: 181
[TRAIN] Acc: 98.870
[TEST] Acc: 87.090
Epoch: 182
[TRAIN] Acc: 98.826
[TEST] Acc: 87.250
Epoch: 183
[TRAIN] Acc: 98.794
[TEST] Acc: 87.170
Epoch: 184
[TRAIN] Acc: 98.814
[TEST] Acc: 86.990
Epoch: 185
[TRAIN] Acc: 98.788
[TEST] Acc: 87.020
Epoch: 186
[TRAIN] Acc: 98.858


In [2]:
lr=0.01
data='cifar100'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar100_sigmoidX_vgg_ProActiv.pth'
resume = False

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 0.996
[TEST] Acc: 1.220
Epoch: 1
[TRAIN] Acc: 0.938
[TEST] Acc: 0.930
Epoch: 2
[TRAIN] Acc: 1.032
[TEST] Acc: 0.840
Epoch: 3
[TRAIN] Acc: 1.078
[TEST] Acc: 0.930
Epoch: 4
[TRAIN] Acc: 0.986
[TEST] Acc: 1.010
Epoch: 5
[TRAIN] Acc: 1.002
[TEST] Acc: 0.990
Epoch: 6
[TRAIN] Acc: 0.932
[TEST] Acc: 0.930
Epoch: 7
[TRAIN] Acc: 0.958
[TEST] Acc: 0.920
Epoch: 8
[TRAIN] Acc: 0.894
[TEST] Acc: 0.960
Epoch: 9
[TRAIN] Acc: 0.978
[TEST] Acc: 1.060
Epoch: 10
[TRAIN] Acc: 1.028
[TEST] Acc: 1.030
Epoch: 11
[TRAIN] Acc: 0.988
[TEST] Acc: 1.060
Epoch: 12
[TRAIN] Acc: 0.994
[TEST] Acc: 1.020
Epoch: 13
[TRAIN] Acc: 0.950
[TEST] Acc: 1.260
Epoch: 14
[TRAIN] Acc: 1.002
[TEST] Acc: 1.080
Epoch: 15
[TRAIN] Acc: 0.972
[TEST] Acc: 1.050
Epoch: 16
[TRAIN] Acc: 0.984
[TEST] Acc: 0.910
Epoch: 17
[TRAIN] Acc: 0.992
[TEST] Acc: 1.050
Epoch: 18
[TRAIN] Acc: 1.012
[TEST] Acc: 1.070
Epoch: 19
[TRAIN] Acc: 0.990
[TEST] Acc: 1.020
Epoch: 20
[TRAIN] Acc: 0.962
[TEST] Acc: 1.240
Epoch: 21
[TRAIN] Acc: 0.996
[TE

KeyboardInterrupt: 

In [2]:
lr=0.01
data='stl10'
root='./data/'
model='vgg'
model_out='./checkpoint/stl10_sigmoidX_vgg_ProActiv.pth'
resume = False

In [10]:
if data == 'stl10':
    epochs = [200, 100, 50]

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 9.680
[TEST] Acc: 9.600
Epoch: 1
[TRAIN] Acc: 10.480
[TEST] Acc: 10.963
Epoch: 2
[TRAIN] Acc: 14.960
[TEST] Acc: 16.462
Epoch: 3
[TRAIN] Acc: 15.120
[TEST] Acc: 18.450
Epoch: 4
[TRAIN] Acc: 17.700
[TEST] Acc: 17.512
Epoch: 5
[TRAIN] Acc: 17.160
[TEST] Acc: 20.075
Epoch: 6
[TRAIN] Acc: 17.760
[TEST] Acc: 17.375
Epoch: 7
[TRAIN] Acc: 17.740
[TEST] Acc: 18.250
Epoch: 8
[TRAIN] Acc: 18.520
[TEST] Acc: 17.988
Epoch: 9
[TRAIN] Acc: 17.820
[TEST] Acc: 16.688
Epoch: 10
[TRAIN] Acc: 18.060
[TEST] Acc: 11.537
Epoch: 11
[TRAIN] Acc: 18.340
[TEST] Acc: 17.225
Epoch: 12
[TRAIN] Acc: 19.900
[TEST] Acc: 20.400
Epoch: 13
[TRAIN] Acc: 19.740
[TEST] Acc: 21.425
Epoch: 14
[TRAIN] Acc: 19.740
[TEST] Acc: 22.812
Epoch: 15
[TRAIN] Acc: 19.900
[TEST] Acc: 21.337
Epoch: 16
[TRAIN] Acc: 19.860
[TEST] Acc: 19.600
Epoch: 17
[TRAIN] Acc: 21.060
[TEST] Acc: 23.925
Epoch: 18
[TRAIN] Acc: 20.760
[TEST] Acc: 22.863
Epoch: 19
[TRAIN] Acc: 21.760
[TEST] Acc: 24.350
Epoch: 20
[TRAIN] Acc: 21.780
[TEST] Acc:

[TEST] Acc: 51.100
Epoch: 167
[TRAIN] Acc: 67.900
[TEST] Acc: 52.775
Epoch: 168
[TRAIN] Acc: 68.860
[TEST] Acc: 53.125
Epoch: 169
[TRAIN] Acc: 68.840
[TEST] Acc: 47.888
Epoch: 170
[TRAIN] Acc: 68.500
[TEST] Acc: 51.850
Epoch: 171
[TRAIN] Acc: 69.540
[TEST] Acc: 51.425
Epoch: 172
[TRAIN] Acc: 68.700
[TEST] Acc: 51.587
Epoch: 173
[TRAIN] Acc: 69.220
[TEST] Acc: 51.550
Epoch: 174
[TRAIN] Acc: 69.080
[TEST] Acc: 47.288
Epoch: 175
[TRAIN] Acc: 67.880
[TEST] Acc: 51.538
Epoch: 176
[TRAIN] Acc: 68.200
[TEST] Acc: 52.062
Epoch: 177
[TRAIN] Acc: 69.440
[TEST] Acc: 52.837
Epoch: 178
[TRAIN] Acc: 69.560
[TEST] Acc: 52.900
Epoch: 179
[TRAIN] Acc: 70.540
[TEST] Acc: 52.462
Epoch: 180
[TRAIN] Acc: 71.120
[TEST] Acc: 52.250
Epoch: 181
[TRAIN] Acc: 69.940
[TEST] Acc: 52.062
Epoch: 182
[TRAIN] Acc: 69.700
[TEST] Acc: 53.763
Epoch: 183
[TRAIN] Acc: 70.240
[TEST] Acc: 52.525
Epoch: 184
[TRAIN] Acc: 71.400
[TEST] Acc: 51.500
Epoch: 185
[TRAIN] Acc: 71.720
[TEST] Acc: 54.300
Epoch: 186
[TRAIN] Acc: 70.920


[TEST] Acc: 59.013
Epoch: 331
[TRAIN] Acc: 88.140
[TEST] Acc: 59.188
Epoch: 332
[TRAIN] Acc: 87.980
[TEST] Acc: 58.850
Epoch: 333
[TRAIN] Acc: 87.980
[TEST] Acc: 59.237
Epoch: 334
[TRAIN] Acc: 87.580
[TEST] Acc: 59.125
Epoch: 335
[TRAIN] Acc: 87.640
[TEST] Acc: 59.212
Epoch: 336
[TRAIN] Acc: 87.860
[TEST] Acc: 59.288
Epoch: 337
[TRAIN] Acc: 87.500
[TEST] Acc: 59.163
Epoch: 338
[TRAIN] Acc: 87.880
[TEST] Acc: 58.625
Epoch: 339
[TRAIN] Acc: 88.180
[TEST] Acc: 58.800
Epoch: 340
[TRAIN] Acc: 87.120
[TEST] Acc: 58.612
Epoch: 341
[TRAIN] Acc: 87.460
[TEST] Acc: 58.888
Epoch: 342
[TRAIN] Acc: 88.340
[TEST] Acc: 59.600
Epoch: 343
[TRAIN] Acc: 87.760
[TEST] Acc: 58.675
Epoch: 344
[TRAIN] Acc: 87.620
[TEST] Acc: 59.337
Epoch: 345
[TRAIN] Acc: 87.260
[TEST] Acc: 58.700
Epoch: 346
[TRAIN] Acc: 87.820
[TEST] Acc: 58.837
Epoch: 347
[TRAIN] Acc: 88.340
[TEST] Acc: 58.938
Epoch: 348
[TRAIN] Acc: 88.060
[TEST] Acc: 59.138
Epoch: 349
[TRAIN] Acc: 88.620
[TEST] Acc: 58.812
